In [1]:
import pandas as pd

from data_processor.ChinaStockDownloader import ChinaStockDownloader

## Download data

In [ ]:
dl = ChinaStockDownloader("2009-01-01", "2022-07-01", index="TICKER_LIST_590")

df = dl.download_fundament(remain_statdate=True)

In [ ]:
df = df[["tic", "date", "statDate"]].rename(columns={"date": "pubDate"})
df

In [ ]:
df.to_csv("./data/590_fund_time.csv")

## Processing

In [2]:
time_table = pd.read_csv("./data/590_fund_time.csv", index_col=0)
time_table.tic = time_table.tic.apply(lambda x: x[3:])
time_table

,tic,pubDate,statDate
0,002287,2009-10-30,2009-09-30
1,002287,2010-04-22,2009-12-31
2,002287,2010-04-22,2010-03-31
3,002287,2010-08-27,2010-06-30
4,002287,2010-10-26,2010-09-30
...,...,...,...
31144,000800,2021-04-30,2021-03-31
31145,000800,2021-08-28,2021-06-30
31146,000800,2021-10-30,2021-09-30
31147,000800,2022-03-31,2021-12-31


In [3]:
fund_path = "./data/fundamental_of_592/"
drop_col = ["inventory_days", "receivable_days", "payable_days"]

In [4]:
sample_fund_df = pd.read_csv("./data/fundamental_of_592/000011.csv", index_col=0)
sample_fund_df.head()

,date,grossMargin,revenue,inventory_days,receivable_days,payable_days,debt_ratio,fixedasset,freecashflow,cashtoemployee,netmargin,num_shareholder,net_equity
0,2008-03-31,35.0331,6.919745e+07,2027.0270,88.1144,140.537665,67.3838,1.208388e+08,-1.031745e+08,4.074940e+07,-12.5229,26652.0,5.549153e+08
1,2008-06-30,30.8971,6.725781e+07,2017.9372,87.1628,133.819329,70.0801,1.201810e+08,-2.891580e+07,8.047956e+07,-7.8092,24558.0,5.559912e+08
2,2008-09-30,30.0654,6.363728e+07,2091.4020,91.2532,134.070811,73.3740,1.033703e+08,-5.550868e+07,1.296675e+08,-4.7032,23744.0,5.461609e+08
3,2008-12-31,45.5351,4.233726e+08,1128.1730,38.9817,114.128085,73.8823,1.040139e+08,-9.811869e+07,1.753009e+08,-3.6516,23762.0,5.714771e+08
4,2009-03-31,44.3054,1.311565e+08,1444.6228,47.1081,155.075889,73.8823,1.009999e+08,-9.811869e+07,4.035210e+07,-3.6516,24262.0,5.899457e+08


In [28]:
tic_list = list(time_table.tic.unique())
tic_with_missing = 0
tic_with_complete_missing_col = 0
complete_missing_col = dict(
    zip(
        sample_fund_df.drop(columns=drop_col).columns,
        [0 for i in range(len(sample_fund_df.drop(columns=drop_col).columns))],
    )
)
time_table = time_table.set_index('tic')

for tic in tic_list:
    complete_missing_flag = False
    fund_df = pd.read_csv(fund_path + tic + ".csv", index_col=0).iloc[6:]
    fund_df = fund_df.drop(columns=drop_col)

    pub_d = time_table.loc[tic]["pubDate"].to_list()
    stat_d = time_table.loc[tic]["statDate"].to_list()
    trans_dict = dict(zip(stat_d, pub_d))
    # print(trans_dict)
    fund_df.date = fund_df.date.map(trans_dict)
    if fund_df.isnull().any().any():
        tic_with_missing += 1
        print(f"\n{tic} has missing data in: ")
        for col in fund_df.columns:
            null_num = fund_df[col].isnull().sum()
            if null_num > 0:
                print(f"{col}: {null_num}")
            if null_num == len(fund_df):
                complete_missing_flag = True
                complete_missing_col[col] += 1
            if fund_df.isnull()[col].all():
                print(f"{tic} {col} missing")
        if complete_missing_flag:
            tic_with_complete_missing_col += 1

print(f"\nThere are {tic_with_missing} stocks have missing data")
print(
    f"{tic_with_complete_missing_col} stocks have columns that are missing completely.\n"
)
print(f"The number of complete missing column: ")
for key, value in complete_missing_col.items():
    print(f"{key}: {value}")



002372 has missing data in: 
date: 2
grossMargin: 1
debt_ratio: 2
fixedasset: 1
freecashflow: 2
netmargin: 2
num_shareholder: 2
net_equity: 1

600809 has missing data in: 
cashtoemployee: 1

601328 has missing data in: 
grossMargin: 50
601328 grossMargin missing
fixedasset: 50
601328 fixedasset missing
freecashflow: 26
netmargin: 50
601328 netmargin missing

002327 has missing data in: 
date: 1
debt_ratio: 1
freecashflow: 1
netmargin: 1
num_shareholder: 1

002351 has missing data in: 
date: 1
debt_ratio: 2
freecashflow: 2
netmargin: 2
num_shareholder: 1

300042 has missing data in: 
date: 1
debt_ratio: 2
freecashflow: 2
netmargin: 2
num_shareholder: 1

300017 has missing data in: 
date: 1
debt_ratio: 1
freecashflow: 1
netmargin: 1
num_shareholder: 1

600403 has missing data in: 
grossMargin: 2
revenue: 2
fixedasset: 3

600036 has missing data in: 
grossMargin: 50
600036 grossMargin missing
fixedasset: 50
600036 fixedasset missing
freecashflow: 27
netmargin: 50
600036 netmargin missing

In [2]:
fund_path = "./data/fundamental_of_592/"
drop_col = ["inventory_days", "receivable_days", "payable_days"]
rename_dict = {
    "Date": "date",
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Volume": "volume",
}

In [3]:
time_table = pd.read_csv("./data/590_fund_time.csv", index_col=0)
time_table.tic = time_table.tic.apply(lambda x: x[3:])

tic_list = list(time_table.tic.unique())

time_table = time_table.set_index('tic')

dataset_df_list = []

for tic in tic_list:
    complete_missing_flag = False
    fund_df = pd.read_csv(fund_path + tic + ".csv", index_col=0).iloc[6:]
    fund_df = fund_df.drop(columns=drop_col)

    pub_d = time_table.loc[tic]["pubDate"].to_list()
    stat_d = time_table.loc[tic]["statDate"].to_list()
    trans_dict = dict(zip(stat_d, pub_d))
    fund_df.date = fund_df.date.map(trans_dict)
    
    fund_df = fund_df.drop_duplicates("date", keep="last", ignore_index=True)

    if fund_df.isnull().any().any():
        for col in fund_df.columns:
            if fund_df[col].isnull().all():  # completely missing
                fund_df[col] = 0.01  # fill 0

    price_df = pd.read_csv(
        f"./data/590data/2010-2021_592_fq/{tic}.csv", index_col=0
    ).rename(columns=rename_dict)

    fund_df.date = pd.to_datetime(fund_df.date)
    price_df.date = pd.to_datetime(price_df.date)

    date_range = list(pd.date_range(fund_df.date.min(), price_df.date.max()))
    merge_df = pd.DataFrame({"date": date_range})

    merge_df = (
        merge_df.merge(fund_df, on="date", how="left")
        .ffill()
        .merge(price_df, on="date", how="left")
    )
    
    merge_df = merge_df.dropna(how="any")
    merge_df['tic'] = tic
    
    dataset_df_list.append(merge_df)

dataset_df = pd.concat(dataset_df_list).sort_values(["date", "tic"], ignore_index=True)
    


In [23]:
dataset_df.to_csv("./data/590_full.csv")

In [4]:
dataset_df.to_pickle("./data/590_full.pkl")

In [5]:
dataset_df = pd.read_pickle("./data/590_full.pkl")

In [6]:
dataset_df

,date,grossMargin,revenue,debt_ratio,fixedasset,freecashflow,cashtoemployee,netmargin,num_shareholder,net_equity,open,close,high,low,volume,tic
0,2010-01-04,53.2092,2.220465e+08,72.6663,8.587061e+07,5.709942e+08,1.400657e+08,11.6486,25726.0,7.316765e+08,34.42,33.94,34.42,33.91,12334.93,000011
1,2010-01-04,33.8821,1.481782e+09,53.2629,8.246742e+09,-1.781245e+08,2.888067e+08,7.8323,104358.0,5.314334e+09,172.17,168.87,172.26,168.43,151089.77,000012
2,2010-01-04,52.8871,1.473787e+08,71.5761,3.308327e+07,-1.136860e+08,2.784275e+07,16.5527,13898.0,4.903319e+08,82.05,80.30,83.26,80.20,32750.34,000014
3,2010-01-04,3.6743,3.942967e+09,25.1157,1.148597e+09,1.084384e+08,3.315917e+08,2.4272,74618.0,3.854678e+09,110.62,111.22,111.81,109.10,129725.42,000021
4,2010-01-04,15.4296,1.224057e+08,68.6222,1.825953e+08,3.802427e+07,4.178006e+07,2.1171,21024.0,1.896136e+08,21.41,21.04,21.49,20.89,2867.09,000025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771422,2022-07-01,0.0100,1.998410e+11,91.5442,1.000000e-02,2.878011e+11,1.128890e+11,0.0100,409698.0,2.614122e+12,11.85,11.87,11.91,11.83,84070.39,601939
1771423,2022-07-01,21.9475,1.962033e+09,12.8985,4.749222e+09,-2.965963e+08,9.693586e+08,7.6525,97019.0,1.258885e+10,13.59,13.41,13.66,13.16,56340.65,601958
1771424,2022-07-01,0.0100,1.494850e+11,91.2836,1.000000e-02,-3.937127e+11,9.399800e+10,0.0100,728790.0,2.350553e+12,6.60,6.60,6.62,6.60,71302.10,601988
1771425,2022-07-01,-0.9188,2.826363e+10,68.4530,1.882654e+11,-8.638582e+09,8.812022e+09,0.7788,206795.0,7.616704e+10,6.60,6.68,6.77,6.60,29816.25,601991
